# *Introduction to few key NLP metrics*

In any scenario, where there is a need to judge how good a machine translation of a given source language is, or judge the performance of a classifier, we need to introduce metrics. Note that, in case of machine translation source and target language can be anything--a PDF, latex code corresponding to a PDF, any human language, etc. Hence these metrics are language-independent. We attempt to introduce few of these metrics here:

+ Levenshtein distance (Edit distance)
+ BLEU (BiLingual Evaluation Understudy)
+ ROUGE (\# Todo)
+ METEOR(Metric for Evaluation of Translation with Explicit ORdering) (\# Todo)
+ Precision
+ Accuracy
+ Recall
+ F1 Score

## Machine Translations related

### Levenshtein distance
 Levenshtein or Edit distance is a fundamental string metric. Given two strings $A$ and $B$, it computes minimum number of insertions, deletions or replacements needed to transform $A$ to $B$. It helps to compute how much different a machine translated string is from the source string. Edit distance has a convenient recursive formula:
 
 $$lev_{a,b}(i,j) = \begin{cases} 
          lev_{a,b}(i+1,j+1) & if \ a[i]=b[j], a,b\neq\phi \\
          1+min\{lev_{a,b}(i,j+1),lev_{a,b}(i+1,j),lev_{a,b}(i+1,j+1)\} & if \ a[i] \neq b[j], a,b\neq \phi \\
          |a[i:]| & if \ b=\phi \\
          |b[j:]| & if \ b\neq\phi, a=\phi
       \end{cases}$$
       
where <mark>$lev_{a,b}(i,j)$ is the levenshtein distance of the string $a$ starting from $i^{th}$ index w.r.t. string $b$ starting from $j^{th}$ index</mark>. 

*EXPLANATION*: In order to compute $lev_{a,b}(i,j)$:
+ If $a[i] = b[j]$, then it is quite clear that we recursively compute the Edit distance of the string $a[i+1:]$ w.r.t. $b[j+1:]
 $
+ If $a[i] \neq b[j]$, then either:
  + we add $b[j]$ in the current position of $a$, so we recursively compute the Edit distance of the string $a[i:]$ w.r.t. $b[j+1:]$ (as $b[j]$ has been matched but didn't exhaust any character of the original $a$) and add $1$ to result(for the $Add$ operation).
  + we replace $a[i]$ with $b[j]$, so we recursively compute the Edit distance of the string $a[i+1:]$ w.r.t. $b[j+1:]$ (since $b[j]$ has been matched, and exhausted one more character of $a$) and add $1$ to result(for the $Replace$ operation).
  + we delete $a[i]$ in hope that the rest of the $a$ string matches with rest of the $b$ string, so we recursively compute Edit distance of $a[i+1:]$ w.r.t. $b[j]$(since character of $a$ has been exhausted without matching $b[j]$)
 
The recursive formula, naturally leads to a bottom up dynamic programming approach, where we take a $(|a|+1)$x$(|b|+1)$ matrix $dp$ with rows indexed by $a$ and column by $b$ such that (we output $dp[0][0]=lev_{a,b}(0,0)$ at the end):
$$ dp[i][j] =\begin{cases}
lev_{a,b}(i,j) & if \ 0\leq i\leq|a|, 0\leq j\leq|b|\\
lev_{a,\phi}(i,j) & if \ j=|b| \\
leb_{\phi,b}(i,j) & if \ i=|a|\\
\end{cases}$$


In [2]:
# runs in O(len(str1)*len(str2))
def levenshtein( str1, str2):
    l1 = len(str1)
    l2 = len(str2)
    matrix = [[-1]*(l2+1) for i in range(0,l1+1)] # initialisation of the (len(str1)+1)x(len(str2)+1) matrix with -1(s)
    
    # base cases
    for i in range(l2+1):
        matrix[l1][i] = l2 - i # initialisation of the bottom row
    
    for j in range(l1+1):
        matrix[j][l2] = l1 - j # initialisation of the rightmost column
        
    # computation starts!
    for i in range(l1-1,-1,-1): # bottom-up fashion--- from second last row to top row, and in each row from second-last cell to starting cell
        for j in range(l2-1,-1,-1):
            if str1[i]==str2[j]:
                matrix[i][j] = matrix[i+1][j+1] # if the characters are equal, move both pointers
            else:
                matrix[i][j] = 1 + min(matrix[i+1][j],matrix[i+1][j+1], matrix[i][j+1]) # else, add 1 (pertaining to either insert, replace or delete) to the minimum of the levenshstein distances of the remaining pieces of the strings
            
    
    return matrix[0][0]



Examples:
+ levenshtein distance of *"abc"* w.r.t *"adc"* is $1$ : *"abc"*  $\xrightarrow[]{\text{replace "b" with "d"}}$ *"adc"*
+ levenshtein distance of *"horse"* w.r.t. *"ros"* is $3$ : *"horse"* $\xrightarrow[]{\text{replace "h" with "r"}}$ *"rorse"* $\xrightarrow[]{\text{delete "r"}}$ *"rose"* $\xrightarrow[]{\text{delete "e"}}$ *"ros"* 

In [4]:
print(levenshtein("abc", "adc"))
print(levenshtein("horse","ros"))

1
3


### BLEU metric



<mark>Definition 1 (*$n-gram$*)</mark>: Generally an $n-gram$ is a sequence of $n$ consecutive words, letters, syllables or base pairs (*for machine translation(MT) purposes they are generally words*) collected from a text corpus(dataset).

Ex: bigrams($2$-grams) of *"This article is on NLP"* are *"This article"*, *"article is"*, *"on NLP"*, etc. 

Given an Machine Translation of a source "sentence", and one or more "references", we need to able to say how close the translaton is w.r.t the references and clearly distinguish a good candidate from a bad one.

<mark>Definition 2(*modified $n-gram$ precision*)</mark> Modified $n-gram$ precision of a candidate sentence w.r.t. one or more references is 

$$ \frac{\sum\limits_{\mathcal{C}\in unique \  n-grams} \ count\_clip(\mathcal{C})}{number \ of \ n-grams}$$, 

where 
$count\_clip(\epsilon) = min(Count, Max\_Ref\_Count)$ , $Count$=number of times the $n-gram$ $\epsilon$ appears in the candidate sentence, $Max\_Ref\_Count$ is the maximum Count of $\epsilon$ over all reference(sentences).

We can calculate the modified $n-gram$ precision using `nltk.translate.bleu_score.modified_precision` method of the NLTK library. (install NLTK using `pip install nltk`)

In [1]:
import nltk
# example 1


reference  = '\dfrac{1}{\sqrt{n} \Sigma_{i=1}^{n} |i \rangle'
ref        = [char for char in reference]

candidate1 = '\frac{1}{\sqrt{n} \Sigma\limits_{i=1}^{n} |i>'
cand       = [char for char in candidate1]
print("Number of unigrams =",len(cand))
references = [ref]
float(nltk.translate.bleu_score.modified_precision(references, cand, n=1)) # modified 1-gram precision = 37/44


Number of unigrams = 44


0.8409090909090909

In [78]:
# example 2

reference1 = 'It is a guide to action that ensures that the military will forever heed Party commands'.split()
reference2 = 'It is the guiding principle which gurantees the military forces always being under the command of the Party'.split()
reference3 = 'It is the practical guide for the army always to heed the directions of the party'.split()

candidate1 = 'It is to insure the troops forever hearing the activity guidebook that party direct'.split()
references = [reference1, reference2, reference3]
print(float(nltk.translate.bleu_score.modified_precision(references, candidate1, n=2))) # modified 2-gram precision = 1/13
float(nltk.translate.bleu_score.modified_precision(references, candidate1, n=1)) # modified 1-gram precision = 8/14

0.07692307692307693


0.5714285714285714

Note: Since we calculate modified $n-gram$ precision for a "sentence"(for  $\LaTeX$ code, it might refer to a single sequence of tokens representing the candidate $\LaTeX$ code for an equation, an image, a table, etc), in order to calculate $n-gram$ precision (denoted $p_n$ )for the whole document(collection of "sentences") we use the following formula:
$$ p_n = \frac{\sum\limits_{\mathcal{C}\in \{ Candidates \}}\sum\limits_{n-gram \in \mathcal{C}}count\_clip(\mathcal{n-gram})}{\sum\limits_{\mathcal{C'}\in \{ Candidates \}}\sum\limits_{n-gram \in \mathcal{C'}}count(\mathcal{n-gram})} $$.

<table><tr><th>Why should we consider a weighted average of $n-gram$ precisions over multiple $n$?</th></tr></table>
It is obvious as $n$-increases(upto a certain point) modified $n-gram$ precision of a corpus(whole document) better relates to translation quality:

+ *low $n$*: Checks the vocabulary of tokens/words used in the candidate, i.e. ensures the candidate doesn't use too many unrelated tokens, but it doesn't enforce token order and one might have a complete garbage candidate with correct tokens all jumbled up.

+ *high $n$*: Checks and ensures proper token order and longer syntactic rules, but may be too harsh on minor deviations from "desired" references.

Hence, we need to average out the modified precision scores for all $n\leq N$(threshold).

But...
<figure>
    <img src="image1.jpg" alt="Alt text" />
    <figcaption><center><mark>Exponentially decaying $n-gram$ scores. Here $H_{i}:Human_{i} $,$S_{j}:System/Machine_{j}$</mark></center></figcaption>
</figure>

as the image shows, modified $n$-gram scores decays rough exponentially with $n$ for human translators("good") as well as machine translators("bad").

Therefore we use a heuristic wherein we take <mark>weighted average of logarithms of $p_n$</mark> for $n\leq N$. (The baseline BLEU uses uniform weights)

<mark>Sentence length</mark>: A candidate should neither be too short, nor too long, already the modified $n-gram$ precisions punish longer candidates that repeat a particular instance of a token too much, or use spurious words, **but**, too short candidates using the right words and orders(ex: a fragment of a reference) are spared, as seen in the following example.  Therefore we deploy a *brevity penalty* on longer sentences.




In [5]:
# example
reference1 = 'It is a guide to action that ensures that the military will forever heed Party commands'.split()
reference2 = 'It is the guiding principle which gurantees the military forces always being under the command of the Party'.split()
reference3 = 'It is the practical guide for the army always to heed the directions of the party'.split()

candidate1 = 'of the'.split() # very short and undesirable candidate
references = [reference1, reference2, reference3]
for i in range(1,3):
    print(float(nltk.translate.bleu_score.modified_precision(references, candidate1, n=i))) # score is still 1.0

1.0
1.0


<mark>Definition $3$(*Brevity penalty*)</mark>: Let $r_s(\mathcal{C}) = |\epsilon| s.t. ||\epsilon|-|\mathcal{C}||=\min\limits_{\mathcal{R}\in \{References\}}||\mathcal{C}|-|\mathcal{R}||$. Call $r_s$ the *best match length*.
Let $r$(*effective refernce corpus length*) be the sum of *best match lengths*($r_s$) of each sentence in the corpus, then $$BP(\text{Brevity Penalty}) =\begin{cases}
1 & if \ c > r\\
e^{1-\frac{r}{c}} & if \ c \leq  r
\end{cases}$$, where $c=|\mathcal{C}|$






In [8]:
reference1 = 'It is a guide to action that ensures that the military will forever heed Party commands'.split()
reference2 = 'It is the guiding principle which gurantees the military forces always being under the command of the Party'.split()
reference3 = 'It is the practical guide for the army always to heed the directions of the party'.split()

candidate1 = 'of the'.split() # very short and undesirable candidate
references = [reference1, reference2, reference3]
cand_len = len(candidate1)
closest_ref_len =  nltk.translate.bleu_score.closest_ref_length(references, cand_len)
nltk.translate.bleu_score.brevity_penalty(closest_ref_len, cand_len) # brevity penalty

0.0009118819655545162

Final BLEU Metric:

$$BLEU = BP. \exp{(\sum\limits_{n=1}^{N}w_n \log{p_n})}$$

In the baseline metric, $w_n=\frac{1}{N}$ and $N=4$


In [11]:
# example
reference  = '\dfrac{1}{\sqrt{n} \Sigma_{i=1}^{n} |i \rangle'
ref        = [char for char in reference]

candidate1 = '\frac{1}{\sqrt{n} \Sigma\limits_{i=1}^{n} |i>'
cand       = [char for char in candidate1]
list_of_references = [[ref]]
hypotheses = [cand]

nltk.translate.bleu_score.corpus_bleu(list_of_references, hypotheses) # BLEU score, by default uniform weight and N=4


0.7447490192819548

### METEOR metric



Sort of a more robust BLEU. Allows synonyms and stemmed words to be matched with the reference word.

\# Todo: Brief description with examples with related code

## Classification related
### Accuracy

<mark> Definition (*Confusion Matrix*)</mark>:The columns of the matrix represent the instances of the predicted classes predicted by a classifier and the rows represent the instances of the actual class. It is used to visualize the performance of a classifier. Note that the role of rows and columns can be inverted in some representations.

Let us work with a binary classifier for now, with two classes namely **Positive** and **Negative**.

\begin{array}{|c|c|c|}
\hline
 & \textbf{Predicted Negative} & \textbf{Predicted Positive} \\
\hline
\textbf{Actual Negative} & TN & FP \\
\hline
\textbf{Actual Positive} & FN & TP \\
\hline
\end{array}

where $TN: True \ Negative$, $TP: True \ Positive$, $FN:False \ Negative$, $FP: False \ Positive$.

Then *Accuracy* is then defined as : $ = {{TP + TN} \over {TP + TN + FP + FN}}$
The following is a logistic regression model which uses artificial train and test set. <mark>You can ignore all the code and simply look at the confusion matrix to calculate accuracy.</mark>

In [13]:
# Import required libraries
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

# Generate a synthetic binary classification dataset
X, y = make_classification(n_samples=100, n_features=2, n_informative=2, 
                            n_redundant=0, random_state=42, n_classes=2)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train a Logistic Regression model
classifier = LogisticRegression()
classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = classifier.predict(X_test)

# Calculate the confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = conf_matrix.ravel()  # Extract values from the confusion matrix

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

# Print results
print("Confusion Matrix:")
print(f"              Predicted Negative   Predicted Positive")
print(f"Actual Negative      {tn}                  {fp}")
print(f"Actual Positive      {fn}                  {tp}")

print("\nAccuracy of the classifier:")
print(f"Accuracy = {accuracy:.2%}")




Confusion Matrix:
              Predicted Negative   Predicted Positive
Actual Negative      15                  1
Actual Positive      0                  14

Accuracy of the classifier:
Accuracy = 96.67%


**From now on, we will cook up arbitary confusion matrices, just to illustrate the metric, without actually spawning the classifier!**

Note: Accuracy alone is not a sufficient metric of performance as shown by the following example:

\begin{array}{|c|c|c|}
\hline
 & \textbf{Predicted Negative} & \textbf{Predicted Positive} \\
\hline
\textbf{Actual Negative} & 0 & 5 \\
\hline
\textbf{Actual Positive} & 0 & 95 \\
\hline
\end{array}

The classfier can have $0.95$ accuracy predicting "*Positive*" always!

### Precision
<mark>Definition</mark>: Fraction of True positives to all predicted positives.
$$ precision={{TP} \over {TP + FP}}$$.

This roughly translates to the classifier's "confidence" in predicting positives--important where False positives are very costly.

### Recall
<mark> Definition</mark>: Fraction of True positives to actual positives.
$$Recall = {{TP} \over {TP + FN}}$$

Recall is crucial in applications where missing positive cases (False Negatives) has serious consequences: Medical diagnosis, Disaster alerts, etc.

### F1-Score
<mark>Definition</mark>: Harmonic mean of precision and recall.
$$F1-score = 2*\frac{Precision*Recall}{Precision+Recall}$$

Precision focuses on the quality of positive predictions (How many of the predicted positives are correct?), whereas Recall focuses on the quantity of true positives identified (How many of the actual positives were found?).
Together, they provide a fuller picture of a classifier's performance and are often combined using the F1-score.



Main Credits @ https://python-course.eu/machine-learning/evaluation-metrics.php and @ https://dl.acm.org/doi/10.3115/1073083.1073135 